In [ ]:
import pandas as pd
sample = pd.read_csv('testset_summaries.csv')

In [ ]:
!pip install rake-nltk

In [ ]:
sample.head(1)

,Unnamed: 0,index,episode_id,episode_filename_prefix,show_filename_prefix,episode_name,episode_description,transcript,summary
0,0,0,spotify:episode:74t5WREXUbhEKNI89CNSkL,74t5WREXUbhEKNI89CNSkL,show_015DbLwcXu2fK7e9jIfbFo,Ep.05 Why We Don't Sell Presets,Today on the podcast we go on a JOURNEY! We ta...,Welcome back to another episode of tuxedo tim...,"This week, we talk about the poopy stuff in ou..."


In [ ]:
podcast = 250
sample_transcript = sample.iloc[podcast]['transcript']
sample_transcript

" In a world where two people God in a podcast for too early for this but their cat at 6:00 in the morning. If you here they start to do a bomb gasps gasps gasps this are you done? No, let me have my fun.  If you hear little sniffling and little like chipmunk noises in the background, that's our kitten Nona if you haven't been following along the latest week or two kitchen.  You're so like you come up with the stupidest jokes when we feel like they're they're bad. They're not good. Like they're not funny. Oh my God. No I so we don't even want to marry me. I don't know so we brought our kitten to the office for the first time today. And so she's kind of exploring around us and underneath me right now. Is she? Yeah, it's always funny when we bring her to a new place. She always crawls out of the  The bags so slowly and then but high in the air like she's on the prowl or something. Oh, she just hit her head on a pole on a c-stand on the season. So we've been very much enjoying kitten pare

In [ ]:
sample.iloc[podcast]['episode_name']

'dealing with distraction'

In [ ]:
from rake_nltk import Rake
r = Rake(max_length=10)
r.extract_keywords_from_text(sample_transcript)
r.get_ranked_phrases()

['one show called crime junkie called prophet',
 'got like six figure incomes like right',
 'like 14 year old daughter literally got',
 'small little stabby make stabbed stabbed',
 'pretty much geared towards phones unless',
 'message without without really bothering',
 'us without cameras even without phones',
 'treasures netflix show ever sick',
 'last night two nights ago',
 'especially like young people teenagers growing',
 'name wrong 80 mick mc',
 'like really feel like every year',
 'certain things like social media make',
 'unavoidable like checking social media',
 'much enjoying kitten parent life',
 'like happy accidents happy accidents',
 'everyone needed something else right away',
 'made like four hundred dollars',
 'also think static images everything',
 'gravitate towards something serious like',
 'first ever life coach dedicated',
 'funny every time seth rogen',
 'probably use like six apps',
 'yeah information people connections news',
 'like seth rogen looks good',
 '

In [ ]:
sample_summary = sample.iloc[podcast]['summary']
sample_summary

"This week we talk about the importance of not being distracted by your phone. We also talk about how to make sure you don't get distracted by distractions, and what it means to be a better person. Thanks for listening!"

In [ ]:
sample_desc = sample.iloc[podcast]['episode_description']
sample_desc

"You're working on a project, decide to look at your phone to read a text - suddenly 2 hours have passed and you've got no work done! It's a situation we all know too well. Today we talk about how distractions have affected our work and our relationship, and how we deal with it. JOIN THE CREATIVE CLUB https://www.patreon.com/acoupleofcreatives FOLLOW US Instagram: @acoupleofcreativespodcast YouTube: A Couple Of Creatives CREDITS Producer: @hollyobee Audio Engineer:\xa0Michael O'Connor SPECIAL THANKS Aidan Yoshimoto, Brian J Duffy, Christopher LeGaspe, Cody Baker, Jake, Jasmin Custeau, Jenna & Joel, Joris Zamoto, Kieran Delport, Lilly May, Loic Philipson, Matthew Palmer, Monique Preciado, Robert Gianelli, Russell K Harrison, Spencer Hart  "

In [ ]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    # def normalize(self, token):
    #     if (token.is_alpha):
    #         return token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False and token.is_alpha:
                    # token = self.normalize(token)
                    # selected_words.append(token)
                    if lower is True:
                        selected_words.append(token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_)
                    else:
                        selected_words.append(token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_)
            sentences.append(selected_words)
        return sentences
    
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=100000, limit=0.0):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        keywords = []
        for i, (key, value) in enumerate(node_weight.items()):
            if value >= limit:
                keywords.append(key)
            # print(key + ' - ' + str(value))
            # if i > number:
            #     break
        return keywords
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
tr4w = TextRank4Keyword()
text = 'A generalized maximum entropy approach to bregman co-clustering and matrix approximation Co-clustering is a powerful data mining technique with varied applications such as text clustering , microarray analysis and recommender systems . Recently , an information-theoretic co-clustering approach applicable to empirical joint probability distributions was proposed . In many situations , co-clustering of more general matrices is desired . In this paper , we present a substantially generalized co-clustering framework wherein any Bregman divergence can be used in the objective function , and various conditional expectation based constraints can be considered based on the statistics that need to be preserved . Analysis of the co-clustering problem leads to the minimum Bregman information principle , which generalizes the maximum entropy principle , and yields an elegant meta algorithm that is guaranteed to achieve local optimality . Our methodology yields new algorithms and also encompasses several previously known clustering and co-clustering algorithms based on alternate minimization.'
tr4w.analyze(text, candidate_pos = ['NOUN','PROPN'], window_size=4, lower=True)
transcript_keywords = tr4w.get_keywords(number=3)
transcript_keywords

['co',
 'clustering',
 'algorithm',
 'function',
 'bregman',
 'information',
 'analysis',
 'divergence',
 'expectation',
 'principle',
 'approach',
 'framework',
 'constraint',
 'matrix',
 'text',
 'technique',
 'mining',
 'application',
 'approximation',
 'datum',
 'problem',
 'paper',
 'statistic',
 'meta',
 'recommender',
 'probability',
 'optimality',
 'system',
 'methodology',
 'minimization',
 'distribution',
 'situation']

In [ ]:
tr4w.analyze(sample_summary, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
generated_summary_keywords = tr4w.get_keywords()
generated_summary_keywords

['week', 'importance', 'phone', 'distraction', 'person', 'thank']

In [ ]:
tr4w.analyze(sample_desc, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
description_keywords = tr4w.get_keywords()
description_keywords

['work',
 'yoshimoto',
 'k',
 'brian',
 'russell',
 'j',
 'gianelli',
 'today',
 'distraction',
 'relationship',
 'duffy',
 'robert',
 'christopher',
 'preciado',
 'aidan',
 'harrison',
 'legaspe',
 'monique',
 'project',
 'phone',
 'text',
 'club',
 'instagram',
 'youtube',
 'couple',
 'creative',
 'credits',
 'producer',
 'audio',
 'engineer',
 'michael',
 'cody',
 'baker',
 'jake',
 'jasmin',
 'custeau',
 'jenna',
 'joel',
 'joris',
 'zamoto',
 'kieran',
 'delport',
 'lilly',
 'loic',
 'philipson',
 'matthew',
 'palmer',
 'thanks',
 'spencer',
 'hart',
 'special',
 'hour',
 'situation']

In [ ]:
summaries = pd.read_csv("summaries.csv")

In [ ]:
summaries.head(1)

,Unnamed: 0,Episode ID,Episode Name,Creator Description,TextRank,LexRank,LSA,BART Base,Fine-Tuned BART,First Five Sentences,Fine-Tuned T5,T5 Base,Coref Resolution + SpanBERT + K-Means,BERT + K-Means,Coref Resolution + BERT + K-Means,SpanBERT + K-Means,SpanBERT + T5,First 15 + T5
0,0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,If you want to start mastering recruiting whic...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful rec...,Hello everybody. What's going on in this Jess...,This is a great episode to start your network ...,If you want to be a network marketing recruiti...,"You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...",This is the #1 network marketing podcast in th...,This is a great podcast to start your network ...


In [ ]:
del summaries["Unnamed: 0"]

In [ ]:
summaries.head(2)

,Episode ID,Episode Name,Creator Description,TextRank,LexRank,LSA,BART Base,Fine-Tuned BART,First Five Sentences,Fine-Tuned T5,T5 Base,Coref Resolution + SpanBERT + K-Means,BERT + K-Means,Coref Resolution + BERT + K-Means,SpanBERT + K-Means,SpanBERT + T5,First 15 + T5
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,If you want to start mastering recruiting whic...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful rec...,Hello everybody. What's going on in this Jess...,This is a great episode to start your network ...,If you want to be a network marketing recruiti...,"You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...",This is the #1 network marketing podcast in th...,This is a great podcast to start your network ...
1,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,A Public Service Announcement from the mrbrown...,Don't be a silent victim of crime. a parody fr...,Have someone escort you home when it's late ab...,But you can touch his life. But you can touch ...,If you only touched you in minor ways never mi...,If you have been molested call the police sho...,"If you have been molested, here are some of th...","Ladies, have you been molested don't be a vic...",This is a Public Service Announcement for Mr. ...,"Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...",--- Send in a voice message: https://anchor.fm...,This is a Public Service Announcement for Mr. ...


In [ ]:
summaries.iloc[0]['LexRank']

"You can do whatever networking events and I'm telling you this is very important, please listen making friends does not mean recruiting and prospecting all  Love them because it allows you to work through people and find referrals. I want to follow and be recruited by people who've got the Instagram stories to prove that their life is good or their Facebook is active and engaged and enthusiastic and energetic and vibrant. "

In [ ]:
def scores(transcript_keywords, summary_keywords):
    total_keywords = len(transcript_keywords)
    total_summary_keywords = len(summary_keywords)
    correct_keywords = 0

    for keyword in summary_keywords:
        if keyword in transcript_keywords:
            correct_keywords += 1
    
    precision = correct_keywords / total_summary_keywords
    recall = correct_keywords / total_keywords
    f1 = (2 *(recall * precision)) / (recall + precision)
    
    return precision, recall, f1

In [ ]:
scores(transcript_keywords, description_keywords)

(0.2830188679245283, 0.026501766784452298, 0.048465266558966075)

#Compare Extraction with actual keywords in itunes Dataset

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'test-281700'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://spotify_asr_dataset/


In [ ]:
bucket_name = 'spotify_asr_dataset'
!gsutil -m cp -r gs://{bucket_name}/dataset.csv /content/
!gsutil -m cp -r gs://{bucket_name}/episodes.csv /content/
!gsutil -m cp -r gs://{bucket_name}/metadata.tsv /content/

Copying gs://spotify_asr_dataset/dataset.csv...
- [1/1 files][  2.9 GiB/  2.9 GiB] 100% Done  41.1 MiB/s ETA 00:00:00           
Operation completed over 1 objects/2.9 GiB.                                      
Copying gs://spotify_asr_dataset/episodes.csv...
/ [1/1 files][  1.2 GiB/  1.2 GiB] 100% Done  38.8 MiB/s ETA 00:00:00           
Operation completed over 1 objects/1.2 GiB.                                      
Copying gs://spotify_asr_dataset/metadata.tsv...
- [1/1 files][112.2 MiB/112.2 MiB] 100% Done                                    
Operation completed over 1 objects/112.2 MiB.                                    


In [ ]:
metadata = pd.read_csv("metadata.tsv", sep='\t')
metadata.head(2)

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj
1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,spotify:episode:000HP8n3hNIfglT2wSI2cA,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA


In [ ]:
itunes_episodes = pd.read_csv('episodes.csv')
itunes_episodes.head(2)

,id,show_id,title,link,guid,subtitle,description,summary,author,audio_url,audio_file_size,audio_mime_type,category,explicit,length,pub_date,keywords
0,0000209c-904f-3794-a8d7-283a70cf993c,8ab10fd5-1acb-330e-be52-beebf19f0703,DJ DA KID QUEST MARCH 14TH MIXSHOW LIVE ON MIX...,https://www.podomatic.com/podcasts/teamosmnetw...,http://teamosmnetwork.podomatic.com/entry/2008...,DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,"<img src=""http://assets.podomatic.net/ts/e7/04...",DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,NaN,http://teamosmnetwork.podOmatic.com/enclosure/...,81099614,audio/mpeg,NaN,1,5571,2008-03-19T17:21:46,"2pac,97,akon,ali,amazing,andre,ashanti,autogra..."
1,000020d5-40ec-351e-aa7a-e8af775de245,16f67b61-d35c-38fa-9bfb-36146d2be45b,Back To Goa - Kameleon part 2 (Archaic Revival),http://max99.libsyn.com/back-to-goa-kameleon-p...,be56007c5636418ce74c58b75dade7f3,Special part 2 of 2 of Back To Goa Pangea spec...,"<p style=""margin: 0.0px 0.0px 0.0px 0.0px; fon...",NaN,NaN,http://traffic.libsyn.com/max99/Back_To_Goa_-_...,189289362,audio/mpeg,NaN,0,1243,2011-10-08T21:15:00,"to,back,trance,pangea,goa,psy,kameleon"


In [ ]:
itunes_shows = pd.read_csv('shows.csv')
itunes_shows.head(2)

,id,feed_url,title,subtitle,description,summary,author,email,link,language,explicit,image,category,subcategory,created_at,last_build_date
0,0002da5f-49a2-31ba-b44c-cdeabdf113cb,http://www.cbc.ca/podcasting/includes/dispatch...,Dispatches from CBC Radio,NaN,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio,podcasting@cbc.ca,http://www.cbc.ca/podcasting,en-ca,0,http://www.cbc.ca/podcasting/images/promo-disp...,News & Politics,Public Radio,2016-11-27T03:15:54,2016-11-27T03:15:54
1,00044845-beb8-35f3-99e2-01dbcfb6eb63,http://everydayissaturday.com/feed/podcast/,Motivation | Inspiration| Success with Entrepr...,Motivation - Inspiration - Success - Mindset -...,Sam Crowley delivers million dollar motivation...,Sam Crowley delivers million dollar motivation...,Sam Crowley,sam@everydayissaturday.com,http://everydayissaturday.com,en-us,0,http://everydayissaturday.com/wp-content/uploa...,Business,Business,2016-11-26T19:11:59,2016-11-21T14:19:34


In [ ]:
itunes_dataset = pd.merge(itunes_episodes, itunes_shows,  how='left', left_on='show_id', right_on ='id')
itunes_dataset.head(2)

,id_x,show_id,title_x,link_x,guid,subtitle_x,description_x,summary_x,author_x,audio_url,audio_file_size,audio_mime_type,category_x,explicit_x,length,pub_date,keywords,id_y,feed_url,title_y,subtitle_y,description_y,summary_y,author_y,email,link_y,language,explicit_y,image,category_y,subcategory,created_at,last_build_date
0,0000209c-904f-3794-a8d7-283a70cf993c,8ab10fd5-1acb-330e-be52-beebf19f0703,DJ DA KID QUEST MARCH 14TH MIXSHOW LIVE ON MIX...,https://www.podomatic.com/podcasts/teamosmnetw...,http://teamosmnetwork.podomatic.com/entry/2008...,DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,"<img src=""http://assets.podomatic.net/ts/e7/04...",DJ THA KID QUEST INTRO 9TH WARD FT JD - ADD ME...,NaN,http://teamosmnetwork.podOmatic.com/enclosure/...,81099614,audio/mpeg,NaN,1,5571,2008-03-19T17:21:46,"2pac,97,akon,ali,amazing,andre,ashanti,autogra...",8ab10fd5-1acb-330e-be52-beebf19f0703,http://teamosmnetwork.podomatic.com/rss2.xml,TEAM OSM NETWORK,IT'S ONLY ONE TEAM ON TOP,NaN,NaN,TEAM OSM NETWORK,NaN,NaN,en-us,0,http://teamosmnetwork.podomatic.com/images/def...,Music,NaN,2016-11-26T18:39:21,2016-11-26T18:39:21
1,000020d5-40ec-351e-aa7a-e8af775de245,16f67b61-d35c-38fa-9bfb-36146d2be45b,Back To Goa - Kameleon part 2 (Archaic Revival),http://max99.libsyn.com/back-to-goa-kameleon-p...,be56007c5636418ce74c58b75dade7f3,Special part 2 of 2 of Back To Goa Pangea spec...,"<p style=""margin: 0.0px 0.0px 0.0px 0.0px; fon...",NaN,NaN,http://traffic.libsyn.com/max99/Back_To_Goa_-_...,189289362,audio/mpeg,NaN,0,1243,2011-10-08T21:15:00,"to,back,trance,pangea,goa,psy,kameleon",16f67b61-d35c-38fa-9bfb-36146d2be45b,http://max99.libsyn.com/rss,Back to Goa,the latest psy trance and goa trance releases,'Back To Goa' has been broadcasting since 2008...,'Back To Goa' has been broadcasting since 2008...,North Soundscape Recordings,NaN,http://max99.libsyn.com,en,0,http://static.libsyn.com/p/assets/7/7/3/e/773e...,Music,Religion & Spirituality,2016-11-25T07:44:25,2016-11-22T10:45:36


In [ ]:
itunes_dataset = itunes_dataset[['id_x', 'show_id', 'category_x', 'category_y', 'subcategory', 'keywords', 'title_x', 'title_y']]

In [ ]:
itunes_dataset.head(2)

,id_x,show_id,category_x,category_y,subcategory,keywords,title_x,title_y
0,0000209c-904f-3794-a8d7-283a70cf993c,8ab10fd5-1acb-330e-be52-beebf19f0703,NaN,Music,NaN,"2pac,97,akon,ali,amazing,andre,ashanti,autogra...",DJ DA KID QUEST MARCH 14TH MIXSHOW LIVE ON MIX...,TEAM OSM NETWORK
1,000020d5-40ec-351e-aa7a-e8af775de245,16f67b61-d35c-38fa-9bfb-36146d2be45b,NaN,Music,Religion & Spirituality,"to,back,trance,pangea,goa,psy,kameleon",Back To Goa - Kameleon part 2 (Archaic Revival),Back to Goa


In [ ]:
combined = pd.merge(metadata, itunes_dataset, how='inner', left_on=['episode_name','show_name'], right_on = ['title_x','title_y'])
combined.head(2)

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,id_x,show_id,category_x,category_y,subcategory,keywords,title_x,title_y
0,spotify:show:7gozmLqbcbr6PScMjc0Zl4,Reply All,"""'A podcast about the internet' that is actual...",Gimlet,['en'],https://feeds.megaphone.fm/replyall,spotify:episode:01ugJcZ5cbSdvhrL92WXYW,"#24 Exit & Return, Part II","This week, we conclude Shulem Deen’s story. I...",23.184100,show_7gozmLqbcbr6PScMjc0Zl4,01ugJcZ5cbSdvhrL92WXYW,24f81324-ccb7-3f72-b454-2914bd2736e1,850417fa-904f-3cfc-8792-751d4b9f04b2,NaN,Technology,Technology,Storytelling,"#24 Exit & Return, Part II",Reply All
1,spotify:show:7gozmLqbcbr6PScMjc0Zl4,Reply All,"""'A podcast about the internet' that is actual...",Gimlet,['en'],https://feeds.megaphone.fm/replyall,spotify:episode:0A6JZqMLAA6Kf8mDQXFVrr,#18 Silence And Respect,"In 2012, a woman named Lindsey Stone posted a ...",26.822517,show_7gozmLqbcbr6PScMjc0Zl4,0A6JZqMLAA6Kf8mDQXFVrr,36ac13ab-e84b-3cf1-9fac-a31de9a192c8,850417fa-904f-3cfc-8792-751d4b9f04b2,NaN,Technology,Technology,Storytelling,#18 Silence And Respect,Reply All


In [ ]:
combined = combined[['show_uri', 'episode_uri', 'show_name', 'show_description', 'episode_name', 'episode_description','publisher', 'category_y', 'subcategory', 'keywords']]

In [ ]:
dataset = pd.read_csv('dataset.csv')
dataset.head(2)

,episode_id,transcript
0,spotify:episode:399kdfMnjw0KYANZU7CQJ0,It's the mother back a podcast. Well that was...
1,spotify:episode:49wcMBeJfaaL6KFFdsWvac,If you haven't heard about anchor is the easi...


In [ ]:
combined = combined.merge(dataset, how='left', left_on='episode_uri', right_on='episode_id')
combined.head(2)

,show_uri,episode_uri,show_name,show_description,episode_name,episode_description,publisher,category_y,subcategory,keywords,episode_id,transcript
0,spotify:show:7gozmLqbcbr6PScMjc0Zl4,spotify:episode:01ugJcZ5cbSdvhrL92WXYW,Reply All,"""'A podcast about the internet' that is actual...","#24 Exit & Return, Part II","This week, we conclude Shulem Deen’s story. I...",Gimlet,Technology,Technology,Storytelling,spotify:episode:01ugJcZ5cbSdvhrL92WXYW,I'm weight. So that livre interview about sho...
1,spotify:show:7gozmLqbcbr6PScMjc0Zl4,spotify:episode:0A6JZqMLAA6Kf8mDQXFVrr,Reply All,"""'A podcast about the internet' that is actual...",#18 Silence And Respect,"In 2012, a woman named Lindsey Stone posted a ...",Gimlet,Technology,Technology,Storytelling,spotify:episode:0A6JZqMLAA6Kf8mDQXFVrr,Before we get started a quick warning that th...


In [ ]:
del combined['episode_id']

In [ ]:
combined.head(2)

,show_uri,episode_uri,show_name,show_description,episode_name,episode_description,publisher,category_y,subcategory,keywords,transcript
0,spotify:show:7gozmLqbcbr6PScMjc0Zl4,spotify:episode:01ugJcZ5cbSdvhrL92WXYW,Reply All,"""'A podcast about the internet' that is actual...","#24 Exit & Return, Part II","This week, we conclude Shulem Deen’s story. I...",Gimlet,Technology,Technology,Storytelling,I'm weight. So that livre interview about sho...
1,spotify:show:7gozmLqbcbr6PScMjc0Zl4,spotify:episode:0A6JZqMLAA6Kf8mDQXFVrr,Reply All,"""'A podcast about the internet' that is actual...",#18 Silence And Respect,"In 2012, a woman named Lindsey Stone posted a ...",Gimlet,Technology,Technology,Storytelling,Before we get started a quick warning that th...


In [ ]:
combined = combined.dropna()

In [ ]:
combined['keywords']

0      Storytelling
1      Storytelling
2      Storytelling
3      Storytelling
4      Storytelling
           ...     
156    Storytelling
158    Storytelling
159    Storytelling
160    Storytelling
163    Storytelling
Name: keywords, Length: 91, dtype: object

In [ ]:
tr4w.analyze(combined.iloc[0]['transcript'], candidate_pos = ['NOUN','PROPN'], window_size=4, lower=True)
keywords = tr4w.get_keywords()
keywords

['internet',
 'people',
 'community',
 'hasidic',
 'lot',
 'thing',
 'asifa',
 'computer',
 'day',
 'week',
 'square',
 'time',
 'new',
 'way',
 'world',
 'guy',
 'video',
 'dean',
 'table',
 'site',
 'story',
 'squarespace',
 'york',
 'website',
 'place',
 'music',
 'hasidim',
 'alex',
 'radio',
 'episode',
 'man',
 'shrimp',
 'pj',
 'orthodox',
 'kid',
 'mail',
 'technology',
 'reply',
 'song',
 'home',
 'picture',
 'filter',
 'friday',
 'board',
 'son',
 'message',
 'shulam',
 'page',
 'signature',
 'stadium',
 'feeling',
 'step',
 'shtetl',
 'moment',
 'thank',
 'forum',
 'coffee',
 'torah',
 'joseph',
 'life',
 'piece',
 'city',
 'stuff',
 'support',
 'faith',
 'shuttle',
 'fact',
 'headline',
 'house',
 'debate',
 'emma',
 'news',
 'de',
 'lor',
 'heretic',
 'group',
 'discussion',
 'end',
 'business',
 'insect',
 'role',
 'bit',
 'rabbi',
 'code',
 'post',
 'space',
 'tap',
 'version',
 'hotline',
 'company',
 'wall',
 'matt',
 'gathering',
 'big',
 'knee',
 'blumberg',
 'jacobs

In [ ]:
tr4w.analyze(combined.iloc[0]['episode_description'], candidate_pos = ['NOUN','PROPN'], window_size=4, lower=True)
keywords = tr4w.get_keywords()
keywords

['internet',
 'story',
 'community',
 'shulem',
 'corner',
 'subscribe',
 'podcast',
 'code',
 'reply',
 'pinnamaneni',
 'week',
 'deen',
 'sruthi',
 'path',
 'family',
 'way',
 'thank',
 'sponsor']

In [ ]:
combined.iloc[0]['episode_description']

"This week, we conclude Shulem Deen’s story.\xa0 In part I, we heard how the internet led him on a path where he was exiled from his community and separated from his family. In part II, Sruthi Pinnamaneni tells the story of how the Hasidic community has tried to block off a corner of the internet for itself, and how this new, informal Hasidic internet might offer Shulem a way back. \xa0 Thanks for listening! Subscribe to our podcast at www.itunes.com/replyall! Sponsors: www.framebridge.com (offer code 'reply') www.stamps.com (offer code 'reply') www.mailchimp.com"